In [18]:
# Has multi-dimensional arrays and matrices.
# Has a large collection of mathematical functions to operate on these arrays.
import numpy as np

# Data manipulation and analysis.
import pandas as pd

# Data visualization tools.
import seaborn as sns

import mesa
from mesa.discrete_space import CellAgent, OrthogonalMooreGrid
from mesa.visualization import SolaraViz, make_plot_component, make_space_component

In [19]:
# Set up an agent similar to 
class DispersalAgent(CellAgent):
    """An agent that for now just occupies a cell in the grid."""

    def __init__(self, model, cell, growth_rate):
        """Initialize the agent."""
        super().__init__(model)
        self.cell = cell
        self.growth_rate = growth_rate

    def hatch(self):
        """Hatch a new agent in an empty cell depending on growth rate."""
        # Check if the agent can hatch based on its growth rate
        if self.random.random() < self.growth_rate:
            # Get the neighbors using the cell's neighborhood
            neighbors = list(self.cell.neighborhood.cells)
            
            # Identify empty cells (cells with no agents)
            empty_cells = [cell for cell in neighbors if len(cell.agents) == 0]
            
            if empty_cells:
                # Choose a random empty cell to hatch into
                new_cell = self.random.choice(empty_cells)
                
                # Create a new agent in the chosen cell
                DispersalAgent(self.model, new_cell, self.growth_rate)


In [20]:
class DispersalModel(mesa.Model):
    """A model with a grid of cells and agents that can occupy them."""

    def __init__(self, n=10, width=100, height=100, seed=None, growth_rate=0.1):
        """Initialize the model instance
        
        Args:            
            n (int): Number of agents to create.
            width (int): Width of the grid.
            height (int): Height of the grid.
            seed (int, optional): Random seed for reproducibility.
        """
        super().__init__(seed=seed)
        self.num_agents = n
        self.grid = OrthogonalMooreGrid(dimensions=(width, height), torus=False, random=self.random)

        # Create cells and agents
        possible_positions = self.grid.all_cells.cells
        # Only select from a specific rectangular area
        min_x,min_y, max_x, max_y = 0, 0, 5, 5
        # filter the possible positions to only those within the specified rectangle
        filtered_possible_positions = [cell for cell in possible_positions
                                       if min_x <= cell.coordinate[0] <= max_x and
                                            min_y <= cell.coordinate[1] <= max_y]
        
        DispersalAgent.create_agents(model=self,
                                     n=self.num_agents, 
                                     growth_rate=growth_rate,
                                     cell=self.random.choices(filtered_possible_positions, k=self.num_agents),)
        
    def step(self):
        """Advance the model by one step."""
        self.agents.shuffle_do("hatch")

In [21]:
# Lets make sure the model works
model = DispersalModel(n=10, width=100, height=100)
for _ in range(1):
    model.step()


# add visualization


Almostall ABMsimulations consist of twophases. First is the initializa
tion phase, where you define the world and its inhabitants, effectively set
ting up the state of the world at time step zero. Let’s write out our model’s
 setup in pseudocode:

 

# Lets set up an agent

In [22]:
# Visualization function for the agents
# for marker type https://matplotlib.org/stable/api/markers_api.html
def agent_portrayal(agent):
    return {
        "color": "tab:red",
        "size": 10,
    }

In [23]:
# Grid Visualization
model_params = {
    "n": {
        "type": "SliderInt",
        "value": 50,
        "label": "Number of agents:",
        "min": 10,
        "max": 100,
        "step": 1,
    },
    "growth_rate": {
        "type": "SliderFloat",
        "value": 0.1,
        "label": "growth_rate:",
        "min": 0,
        "max": 1,
        "step": 0.1,
    },
    "width": 100,  # Width of the grid
    "height": 100,  # Height of the grid
}  # In a notebook environment

In [27]:
# Create initial model instance
model = DispersalModel(width=100, height=100)  # keyword arguments
model.step()
SpaceGraph = make_space_component(agent_portrayal)

page = SolaraViz(
    model,
    components=[SpaceGraph],
    model_params=model_params,
    name="Dispersal",
)


In [26]:
page

Cannot show ipywidgets in text